In [8]:
import cv2
import mediapipe as mp
from cvzone.HandTrackingModule import HandDetector
import socket
import json

cap = cv2.VideoCapture(0) # uncomment if you have webcam
# cap = cv2.VideoCapture("http://10.0.0.115:4747/video") # ONLY OPENS UP WHEN ONE INSTANCE OF THIS IS OPEN! DO NOT OPEN IN BROWSER!
IMG_HEIGHT = 720
IMG_WIDTH = 1280 
cap.set(3, IMG_WIDTH)
cap.set(4, IMG_HEIGHT)
if not cap.isOpened():
    raise IOError("Cannot open webcam")
    
# DRAW BOTH HANDS ON SCREEN
detector = HandDetector(maxHands=1, detectionCon=0.8)

# Comms
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
serverAddressPort = ("127.0.0.1", 5052)

while(True):
    success, img = cap.read()
    # flipping image to make everything easier to control (like a mirror)

    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img, flipType=False)

    data = []
    
    if hands:
        hand = hands[0]

        # get list of landmarks, add to data object
        lmList = hand["lmList"]

        for lm in lmList:
            data.extend([lm[0], IMG_HEIGHT - lm[1], lm[2]])

        sock.sendto(str.encode(str(data)), serverAddressPort)
    # if hands:
    #     hand = hands[0]
    #     # get list of landmarks
    #     lmList = hand['lmList']

    #     for lm in lmList:
    #         data.extend([lm[0], IMG_HEIGHT - lm[1], lm[2]])
    #     sock.sendto(str.encode(str(data)), serverAddressPort)
    
    cv2.imshow('Image', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [7]:
print(hand['lmList'])
print(hand)

[[479, 341, 0], [551, 317, -23], [612, 267, -32], [630, 206, -40], [622, 157, -49], [593, 185, -18], [622, 118, -41], [640, 79, -60], [659, 51, -72], [554, 165, -22], [581, 82, -42], [602, 34, -59], [622, 0, -71], [514, 161, -30], [540, 82, -51], [558, 35, -66], [578, 1, -75], [470, 172, -39], [476, 109, -61], [486, 68, -70], [499, 33, -76]]
{'lmList': [[479, 341, 0], [551, 317, -23], [612, 267, -32], [630, 206, -40], [622, 157, -49], [593, 185, -18], [622, 118, -41], [640, 79, -60], [659, 51, -72], [554, 165, -22], [581, 82, -42], [602, 34, -59], [622, 0, -71], [514, 161, -30], [540, 82, -51], [558, 35, -66], [578, 1, -75], [470, 172, -39], [476, 109, -61], [486, 68, -70], [499, 33, -76]], 'bbox': (470, 0, 189, 341), 'center': (564, 170), 'type': 'Left'}
